# EE599 Homework 2 — Basic Statistical Tests (Submission)
Name: Hari Krishna Koneti

This notebook implements the required statistical functions and answers all **Application of Functions** questions using the provided sample data.



---

# Dataset & Device Specification

## Concurrent Dataset (actigraph_and_fitbit directory)
- **ActiGraph (Participant 1):**
  - Files used:
    - 1_AG_week1_minute_long.csv
    - 1_AG_week2_minute_long.csv
  - Device: ActiGraph wGT3X+ (hip-worn)

- **ActiGraph (Participant 2):**
  - Files used:
    - 2_AG_week1.csv
    - 2_AG_week2.csv
  - Device: ActiGraph wGT3X+

- **Fitbit (Participant 1):**
  - File used:
    - 1_FB_minuteSteps_minute_long_converted.csv
  - Device: Fitbit (wrist-worn)
  - Original format: Fitabase minuteSteps export (wide per hour), converted to minute-level long format.

## Multiyear Dataset (multiyear directory)
- File used:
  - dailySteps.csv
- Device: Fitbit
- Used for seasonality (Repeated Measures ANOVA).
---



## Imports
Only basic CSV handling + SciPy distribution lookups (allowed for p-values).

In [ ]:
import math
import numpy as np
import pandas as pd
from scipy import stats

## Functions 
### Harmonic Mean, Sample Std Dev, Pooled Std Dev, T-test, ANOVA, Repeated-Measures ANOVA

In [ ]:
def arithmetic_mean(x):
    x = [float(v) for v in x]
    return sum(x) / len(x)

def harmonic_mean(x, drop_zeros=False):
    """H = n / sum(1/x_i). Harmonic mean is undefined if any x_i == 0."""
    x = [float(v) for v in x]
    if drop_zeros:
        x = [v for v in x if v != 0.0]
    if len(x) == 0:
        raise ValueError("Empty dataset after cleaning.")
    if any(v == 0.0 for v in x):
        raise ValueError("Harmonic mean undefined for zero values.")
    n = len(x)
    return n / sum(1.0/v for v in x)

def sample_std(x):
    """Sample standard deviation: sqrt( sum((x_i-mean)^2) / (n-1) )."""
    x = [float(v) for v in x]
    n = len(x)
    if n < 2:
        raise ValueError("Need at least 2 values.")
    mu = arithmetic_mean(x)
    ss = sum((v - mu)**2 for v in x)
    return math.sqrt(ss / (n - 1))

def pooled_std(*pairs):
    """Pooled SD for any number of (sigma_i, n_i) pairs."""
    if len(pairs) < 2:
        raise ValueError("Provide at least two (sigma, n) pairs.")
    num = 0.0
    den = 0
    for sigma_i, n_i in pairs:
        if n_i < 2:
            raise ValueError("Each group must have n >= 2.")
        num += (n_i - 1) * (sigma_i**2)
        den += (n_i - 1)
    return math.sqrt(num / den)

def t_test_pooled(
    x=None, y=None,
    mu1=None, sigma1=None, n1=None,
    mu2=None, sigma2=None, n2=None,
    mean_type="arithmetic"
):
    """Two-sample pooled t-test. Accepts raw datasets OR summary stats."""
    if x is not None and y is not None:
        x = [float(v) for v in x]
        y = [float(v) for v in y]
        if mean_type == "harmonic":
            mu1 = harmonic_mean(x, drop_zeros=True)
            mu2 = harmonic_mean(y, drop_zeros=True)
        else:
            mu1 = arithmetic_mean(x)
            mu2 = arithmetic_mean(y)
        sigma1 = sample_std(x)
        sigma2 = sample_std(y)
        n1 = len(x)
        n2 = len(y)
    else:
        if None in (mu1, sigma1, n1, mu2, sigma2, n2):
            raise ValueError("Provide either (x,y) OR all of (mu1,sigma1,n1,mu2,sigma2,n2).")

    df = n1 + n2 - 2
    sp = math.sqrt(((n1 - 1)*sigma1**2 + (n2 - 1)*sigma2**2) / df)
    t = (mu1 - mu2) / (sp * math.sqrt((1/n1) + (1/n2)))

    # p-value lookup (allowed)
    p = 2 * stats.t.sf(abs(t), df)
    return t, df, p

def anova_one_way(*groups):
    """One-way ANOVA: compute F from scratch; use F distribution for p-value."""
    if len(groups) < 3:
        raise ValueError("Need 3+ groups for ANOVA.")
    groups = [np.array([float(v) for v in g]) for g in groups]
    ns = [len(g) for g in groups]
    if any(n == 0 for n in ns):
        raise ValueError("Empty group found.")
    m = len(groups)
    N = sum(ns)

    grand_mean = sum(g.sum() for g in groups) / N
    ss_between = sum(n * (g.mean() - grand_mean)**2 for n, g in zip(ns, groups))
    ss_total = sum(((g - grand_mean)**2).sum() for g in groups)
    ss_within = ss_total - ss_between

    df_between = m - 1
    df_within = N - m

    ms_between = ss_between / df_between
    ms_within = ss_within / df_within

    F = ms_between / ms_within
    p = stats.f.sf(F, df_between, df_within)  # allowed
    return F, df_between, df_within, p

def rmanova(matrix):
    """Repeated-measures ANOVA on n subjects x k conditions."""
    X = np.array(matrix, dtype=float)
    n, k = X.shape
    grand = X.mean()
    subj_means = X.mean(axis=1)
    cond_means = X.mean(axis=0)

    ss_subjects = k * np.sum((subj_means - grand)**2)
    ss_conditions = n * np.sum((cond_means - grand)**2)
    ss_total = np.sum((X - grand)**2)
    ss_error = ss_total - ss_subjects - ss_conditions

    df_conditions = k - 1
    df_subjects = n - 1
    df_error = df_conditions * df_subjects

    ms_conditions = ss_conditions / df_conditions
    ms_error = ss_error / df_error

    F = ms_conditions / ms_error
    p = stats.f.sf(F, df_conditions, df_error)  # allowed
    return F, df_conditions, df_error, p


## Data loading helpers
ActiGraph raw files contain metadata rows. Fitbit minuteSteps export is wide per hour (measure00..measure59).

In [ ]:
def read_actigraph_raw_daily(path, epoch_seconds=60):
    """Read ActiGraph raw export (with metadata) and return daily step totals."""
    with open(path, "r", errors="ignore") as f:
        lines = f.readlines()

    data_start = None
    start_date = None
    start_time = None

    for i, line in enumerate(lines):
        s = line.strip()
        if not s:
            continue
        if s.lower().startswith("start date"):
            start_date = s.split()[-1]
        if s.lower().startswith("start time"):
            start_time = s.split()[-1]
        if s[0].isdigit():
            data_start = i
            break

    if data_start is None:
        raise ValueError("Could not find numeric data start.")

    df = pd.read_csv(path, skiprows=data_start, header=None, sep=r"\t|,", engine="python")
    if df.shape[1] == 1:
        df = df[0].str.split(r"\s+", expand=True)

    df = df.apply(pd.to_numeric, errors="coerce")
    df.columns = ["Axis1","Axis2","Axis3","Steps"] + [f"col{i}" for i in range(4, df.shape[1])]

    start = pd.to_datetime(f"{start_date} {start_time}")
    df["timestamp"] = start + pd.to_timedelta(np.arange(len(df)) * epoch_seconds, unit="s")
    return df.groupby(df["timestamp"].dt.date)["Steps"].sum()

def read_fitbit_minute_long(path):
    """Read converted Fitbit minute-long CSV with columns: timestamp, Steps."""
    df = pd.read_csv(path, parse_dates=["timestamp"])
    df["Steps"] = pd.to_numeric(df["Steps"], errors="coerce")
    df = df.dropna(subset=["timestamp","Steps"])
    return df


## Application of  Functions
The first four use the concurrently measured data (ActiGraph + Fitbit). The last uses multiyear Fitbit daily totals.

### 1) Daily Steps (ActiGraph)
**Question:** How many steps/day, on average do the subjects walk? Use harmonic and arithmetic mean. Are they different? Why?

- Participant 1: arithmetic mean = **9,943.65**, harmonic mean = **8,578.75** steps/day
- Participant 2: arithmetic mean = **6,750.24**, harmonic mean = **5,314.89** steps/day
- Combined (all subject-days, n=37): arithmetic mean = **8,476.41**, harmonic mean = **6,690.89** steps/day

**Why different?** Harmonic mean weights smaller daily totals more heavily, so it is usually **lower** than the arithmetic mean when the distribution is skewed or has low-activity days.

### 2) Group Variance (ActiGraph pooled SD across subjects)
**Question:** What is the variance of the group? (pooled standard deviation)

- Participant 1 daily SD = **3,571.03** (n=20 days)
- Participant 2 daily SD = **2,925.92** (n=17 days)
- **Pooled SD** across subjects = **3,291.85** steps/day


### 3) Comparing the Devices (Fitbit vs ActiGraph, daily totals)
**Question:** Does Fitbit report the same step measures as ActiGraph? (t-test)

- Common days aligned (participant 1): **19**
- ActiGraph mean = **10,045.37** steps/day
- Fitbit mean = **13,344.74** steps/day
- t = **-2.056**, df = **36**, p = **0.0471**

**Interpretation:** If p < 0.05, reject the null (devices differ). Here, p is shown above. Note: only participant 1 had Fitbit steps available in the provided files, so this comparison is limited to that subject.

### 4) Weekend Warriors (One-way ANOVA by day-of-week, ActiGraph daily steps)
**Question:** Are the subjects equally active across each day of the week?

- F = **1.282**, df_between = **6**, df_within = **30**, p = **0.2953**

**Interpretation:** If p < 0.05, steps differ by day-of-week. Otherwise, no evidence of weekday effects in this sample.

### 5) Seasonality (Repeated Measures ANOVA on multiyear daily steps)
**Question:** Across the two years, were all months traveled equally? (repeated measures ANOVA)

Approach used here:
- Treat **year** as the 'subject' (2013 vs 2014)
- Treat **month** as the condition
- Use months available in both years (Jan–Oct)

- F = **0.474**, df_conditions = **9**, df_error = **9**, p = **0.8596**

**Interpretation:** If p < 0.05, average steps differ by month (seasonality). Because only 2 'subjects' (years) are available, this test has low power; results should be interpreted cautiously.

## Reproducible computation cells

In [ ]:
# ActiGraph participant 1 (minute-long) -> daily totals
ag1w1 = pd.read_csv("/mnt/data/1_AG_week1_minute_long.csv", parse_dates=["timestamp"])
ag1w2 = pd.read_csv("/mnt/data/1_AG_week2_minute_long.csv", parse_dates=["timestamp"])
ag1 = pd.concat([ag1w1, ag1w2], ignore_index=True)
ag1["Steps"] = pd.to_numeric(ag1["Steps"], errors="coerce")
ag1 = ag1.dropna(subset=["Steps","timestamp"])
ag1_daily = ag1.groupby(ag1["timestamp"].dt.date)["Steps"].sum()

# ActiGraph participant 2 (raw with metadata) -> daily totals
ag2_daily = pd.concat([
    read_actigraph_raw_daily("/mnt/data/2_AG_week1.csv"),
    read_actigraph_raw_daily("/mnt/data/2_AG_week2.csv")
]).groupby(level=0).sum()

ag1_daily.head(), ag2_daily.head()


In [ ]:
# Fitbit converted minute-long -> daily totals
fb = read_fitbit_minute_long("/mnt/data/1_FB_minuteSteps_minute_long_converted.csv")
fb_daily = fb.groupby(fb["timestamp"].dt.date)["Steps"].sum()

# Align device days (participant 1)
common_dates = sorted(set(ag1_daily.index).intersection(set(fb_daily.index)))
ag_common = [ag1_daily[d] for d in common_dates]
fb_common = [fb_daily[d] for d in common_dates]

len(common_dates), common_dates[:3]


In [ ]:
# Device comparison t-test (pooled)
t, df, p = t_test_pooled(x=ag_common, y=fb_common, mean_type="arithmetic")
t, df, p
